#### Loading dataset

In [ ]:
#import required packages
import pandas as pd
import numpy as np

In [ ]:
#load dataset
Data=pd.read_csv("/content/Online Retail .csv")

In [ ]:
Data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


#### Data preprocessing

In [ ]:
#filter data for France
FranceData = Data[Data['Country']=='France']
FranceData.reset_index(inplace = True, drop = True)
FranceData.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12/1/2010 8:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,12/1/2010 8:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12/1/2010 8:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,12/1/2010 8:45,0.65,12583.0,France


In [ ]:
#save required columns into new dataframe and also drop records InvoiceNo starts with 'C'
I_Data = FranceData[['InvoiceNo','Description']]
I_Data = I_Data[~I_Data['InvoiceNo'].str.startswith('C')]
I_Data.head()

,InvoiceNo,Description
0,536370,ALARM CLOCK BAKELIKE PINK
1,536370,ALARM CLOCK BAKELIKE RED
2,536370,ALARM CLOCK BAKELIKE GREEN
3,536370,PANDA AND BUNNIES STICKER SHEET
4,536370,STARS GIFT TAPE


In [ ]:
#Replace spaces in item name with '-'
I_Data=I_Data.replace(' ', '_', regex=True)
I_Data.head()

,InvoiceNo,Description
0,536370,ALARM_CLOCK_BAKELIKE_PINK
1,536370,ALARM_CLOCK_BAKELIKE_RED_
2,536370,ALARM_CLOCK_BAKELIKE_GREEN
3,536370,PANDA_AND_BUNNIES_STICKER_SHEET
4,536370,STARS_GIFT_TAPE_


In [ ]:
#verify for any null values in dataframe
I_Data.isna().sum()

InvoiceNo      0
Description    0
dtype: int64

In [248]:
pd.set_option('display.max_colwidth', None)

In [249]:
#Group records based on 'InvoiceNo', this is to have all items(Descriptions) of same InvoiceNo together
G_Data=I_Data.groupby('InvoiceNo')['Description'].apply(' '.join).reset_index()
G_Data.head()

,InvoiceNo,Description
0,536370,ALARM_CLOCK_BAKELIKE_PINK ALARM_CLOCK_BAKELIKE_RED_ ALARM_CLOCK_BAKELIKE_GREEN PANDA_AND_BUNNIES_STICKER_SHEET STARS_GIFT_TAPE_ INFLATABLE_POLITICAL_GLOBE_ VINTAGE_HEADS_AND_TAILS_CARD_GAME_ SET/2_RED_RETROSPOT_TEA_TOWELS_ ROUND_SNACK_BOXES_SET_OF4_WOODLAND_ SPACEBOY_LUNCH_BOX_ LUNCH_BOX_I_LOVE_LONDON CIRCUS_PARADE_LUNCH_BOX_ CHARLOTTE_BAG_DOLLY_GIRL_DESIGN RED_TOADSTOOL_LED_NIGHT_LIGHT _SET_2_TEA_TOWELS_I_LOVE_LONDON_ VINTAGE_SEASIDE_JIGSAW_PUZZLES MINI_JIGSAW_CIRCUS_PARADE_ MINI_JIGSAW_SPACEBOY MINI_PAINT_SET_VINTAGE_ POSTAGE
1,536852,PICTURE_DOMINOES MINI_JIGSAW_SPACEBOY MINI_JIGSAW_DOLLY_GIRL CHARLOTTE_BAG_DOLLY_GIRL_DESIGN VINTAGE_HEADS_AND_TAILS_CARD_GAME_ POLKADOT_RAIN_HAT_ POSTAGE
2,536974,EDWARDIAN_PARASOL_BLACK EDWARDIAN_PARASOL_PINK EDWARDIAN_PARASOL_RED RED__HARMONICA_IN_BOX_ LUNCH_BOX_I_LOVE_LONDON LUNCH_BOX_WITH_CUTLERY_RETROSPOT_ SET_OF_SALT_AND_PEPPER_TOADSTOOLS TEA_PARTY_BIRTHDAY_CARD ASSORTED_COLOUR_MINI_CASES RED_RETROSPOT_MINI_CASES BIG_DOUGHNUT_FRIDGE_MAGNETS POPPY'S_PLAYHOUSE_KITCHEN RIBBON_REEL_POLKADOTS_ LUNCH_BAG_RED_RETROSPOT LUNCH_BAG_WOODLAND POSTAGE
3,537065,HOT_WATER_BOTTLE_BABUSHKA_ BREAD_BIN_DINER_STYLE_RED_ SET_OF_SALT_AND_PEPPER_TOADSTOOLS ROSE_COTTAGE_KEEPSAKE_BOX_ CLASSICAL_ROSE_SMALL_VASE RED_RETROSPOT_PURSE_ POLKADOT_RAIN_HAT_ RAIN_PONCHO_RETROSPOT RED_RETROSPOT_MINI_CASES ASSORTED_COLOUR_MINI_CASES DECORATIVE_CATS_BATHROOM_BOTTLE SET/10_RED_POLKADOT_PARTY_CANDLES BASKET_OF_TOADSTOOLS PARTY_BUNTING ALARM_CLOCK_BAKELIKE_RED_ ALARM_CLOCK_BAKELIKE_GREEN ALARM_CLOCK_BAKELIKE_PINK ALARM_CLOCK_BAKELIKE_ORANGE 5_HOOK_HANGER_RED_MAGIC_TOADSTOOL GUMBALL_COAT_RACK FIVE_HEART_HANGING_DECORATION PINK__HONEYCOMB_PAPER_BALL_ LUNCH_BAG_RED_RETROSPOT LUNCH_BAG_WOODLAND LUNCH_BAG_CARS_BLUE STRAWBERRY_LUNCH_BOX_WITH_CUTLERY ROUND_SNACK_BOXES_SET_OF4_WOODLAND_ LUNCH_BOX_WITH_CUTLERY_RETROSPOT_ CERAMIC_STRAWBERRY_TRINKET_TRAY RETROSPOT_LARGE_MILK_JUG RED_RETROSPOT_BUTTER_DISH REGENCY_CAKESTAND_3_TIER GREEN_3_PIECE_POLKADOT_CUTLERY_SET RED_3_PIECE_RETROSPOT_CUTLERY_SET BLUE_3_PIECE_POLKADOT_CUTLERY_SET CHERRY_BLOSSOM__DECORATIVE_FLASK CHILDS_BREAKFAST_SET_SPACEBOY_ CHILDS_BREAKFAST_SET_DOLLY_GIRL_ CHILDS_BREAKFAST_SET_CIRCUS_PARADE RETROSPOT_LAMP PACK_OF_12_RED_RETROSPOT_TISSUES_ PACK_OF_12_WOODLAND_TISSUES_ PACK_OF_12_SUKI_TISSUES_ PACK_OF_12_SPACEBOY_TISSUES PACK_OF_12_CIRCUS_PARADE_TISSUES_ BLUE_GEISHA_GIRL_ GREEN_GEISHA_GIRL_ PLASTERS_IN_TIN_WOODLAND_ANIMALS PLASTERS_IN_TIN_SPACEBOY PARTY_TIME_PENCIL_ERASERS SET_OF_6_3D_KIT_CARDS_FOR_KIDS HAND_WARMER_RED_RETROSPOT DINOSAUR_KEYRINGS_ASSORTED CERAMIC_STRAWBERRY_MONEY_BOX CERAMIC_CHERRY_CAKE_MONEY_BANK PAPERWEIGHT_CHILDHOOD_MEMORIES RED_GINGHAM_TEDDY_BEAR_ NURSE'S_BAG_SOFT_TOY TOOL_BOX_SOFT_TOY_ TRADITIONAL_WOODEN_CATCH_CUP_GAME_ SET_OF_6_SOLDIER_SKITTLES PINK_PAINTED_KASHMIRI_CHAIR POSTAGE
4,537463,"JAM_MAKING_SET_PRINTED SET/4_SKULL_BADGES ROUND_SNACK_BOXES_SET_OF4_WOODLAND_ SET/10_BLUE_POLKADOT_PARTY_CANDLES SET/10_RED_POLKADOT_PARTY_CANDLES BLACK_RECORD_COVER_FRAME RECORD_FRAME_7""_SINGLE_SIZE_ ROBOT_BIRTHDAY_CARD TEA_PARTY_BIRTHDAY_CARD FANCY_FONT_BIRTHDAY_CARD,_ BOOM_BOX_SPEAKER_BOYS RED_RETROSPOT_CUP IVORY_DINER_WALL_CLOCK LUNCH_BOX_WITH_CUTLERY_RETROSPOT_ STRAWBERRY_LUNCH_BOX_WITH_CUTLERY SNOWSTORM_PHOTO_FRAME_FRIDGE_MAGNET BIG_DOUGHNUT_FRIDGE_MAGNETS WOODLAND_CHARLOTTE_BAG WOODLAND__STICKERS PLASTERS_IN_TIN_WOODLAND_ANIMALS PACK_OF_72_RETROSPOT_CAKE_CASES OFFICE_MUG_WARMER_POLKADOT OFFICE_MUG_WARMER_PINK COFFEE_MUG_CAT_+_BIRD_DESIGN COFFEE_MUG_DOG_+_BALL_DESIGN TABLECLOTH_RED_APPLES_DESIGN_ DOORMAT_RESPECTABLE_HOUSE RED_RETROSPOT_CHILDRENS_UMBRELLA PINK_POLKADOT_CHILDRENS_UMBRELLA BOX_OF_24_COCKTAIL_PARASOLS GUMBALL_COAT_RACK METAL_SIGN_TAKE_IT_OR_LEAVE_IT_ SET_OF_SALT_AND_PEPPER_TOADSTOOLS PACK_OF_20_NAPKINS_RED_APPLES TEA_BAG_PLATE_RED_RETROSPOT LOVELY_BONBON_STICKER_SHEET CANDY_SHOP__STICKER_SHEET SKULLS__WATER_TRANSFER_TATTOOS_ SET_OF_2_TEA_TOWELS_APPLE_AND_PEARS RED_TOADSTOOL_LED_NIGHT_LIGHT POSTAGE"


In [213]:
#Count the total number of transactions using InvoiceNo column.
total_trans = G_Data['InvoiceNo'].nunique()
total_trans

298

In [ ]:
#Save grouped items into a new dataframe for further processing
df_item = G_Data[['Description']]
df_item.head()

,Description
0,ALARM_CLOCK_BAKELIKE_PINK ALARM_CLOCK_BAKELIKE...
1,PICTURE_DOMINOES MINI_JIGSAW_SPACEBOY MINI_JIG...
2,EDWARDIAN_PARASOL_BLACK EDWARDIAN_PARASOL_PINK...
3,HOT_WATER_BOTTLE_BABUSHKA_ BREAD_BIN_DINER_STY...
4,JAM_MAKING_SET_PRINTED SET/4_SKULL_BADGES ROUN...


In [ ]:
#save the processed data into textfile
df_item.to_csv('item.txt', sep='\t', header=None, index=False)

#### Aprior algorithm

In [ ]:
#install required packages
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 19.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=7a8a29c252391b3d9b07f628f4a8dd465a744594e2482fc43238e3a46fca9661
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
#import required packages
import itertools
import pyspark

In [ ]:
#load spark content
sc=pyspark.SparkContext("local","Apriori")

In [ ]:
#load preprocessed data from text file
text_input = sc.textFile('item.txt')

In [ ]:
text_input

item.txt MapPartitionsRDD[18] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
from operator import add
no_of_baskets = text_input.map(lambda line: line.split()).count()
no_of_baskets

298

In [ ]:
txt_RDD = text_input.map(lambda line:line.split())

In [ ]:
#split data and create RDD with item and frequency
individual_counts = text_input.flatMap(lambda line: line.split()).map(lambda x: (x,1)).reduceByKey(add)

In [ ]:
individual_counts.take(10)

[('ALARM_CLOCK_BAKELIKE_PINK', 28),
 ('ALARM_CLOCK_BAKELIKE_RED_', 25),
 ('ALARM_CLOCK_BAKELIKE_GREEN', 28),
 ('PANDA_AND_BUNNIES_STICKER_SHEET', 2),
 ('STARS_GIFT_TAPE_', 13),
 ('INFLATABLE_POLITICAL_GLOBE_', 8),
 ('VINTAGE_HEADS_AND_TAILS_CARD_GAME_', 7),
 ('SET/2_RED_RETROSPOT_TEA_TOWELS_', 8),
 ('ROUND_SNACK_BOXES_SET_OF4_WOODLAND_', 47),
 ('SPACEBOY_LUNCH_BOX_', 44)]

In [215]:
#make copy of individual_counts
temp_ = individual_counts

In [246]:
#Exclude items which fail min support of 0.08
#Support of individual item is (freq of item) / total transactions(invoices)
#Total transactions calculated above and stored in "total_trans"

one_item_support = temp_.filter(lambda x: x[1]/total_trans>=0.08)

In [244]:
one_item_support.take(10)

[('ALARM_CLOCK_BAKELIKE_PINK', 28),
 ('ALARM_CLOCK_BAKELIKE_GREEN', 28),
 ('ROUND_SNACK_BOXES_SET_OF4_WOODLAND_', 47),
 ('SPACEBOY_LUNCH_BOX_', 44),
 ('RED_TOADSTOOL_LED_NIGHT_LIGHT', 52),
 ('MINI_PAINT_SET_VINTAGE_', 32),
 ('POSTAGE', 225),
 ('LUNCH_BOX_WITH_CUTLERY_RETROSPOT_', 49),
 ('TEA_PARTY_BIRTHDAY_CARD', 31),
 ('RED_RETROSPOT_MINI_CASES', 44)]

In [227]:
#filtering to only include items that appear more than 2 times
freq_items_counts = one_item_support.filter(lambda x: x[1]>=2).sortBy(lambda x: x[0])
freq_items_counts.take(10)

[('DOLLY_GIRL_LUNCH_BOX', 35),
 ('JUMBO_BAG_RED_RETROSPOT', 31),
 ('LUNCH_BAG_APPLE_DESIGN', 43),
 ('LUNCH_BAG_RED_RETROSPOT', 50),
 ('LUNCH_BAG_SPACEBOY_DESIGN_', 42),
 ('LUNCH_BAG_WOODLAND', 39),
 ('LUNCH_BOX_WITH_CUTLERY_RETROSPOT_', 49),
 ('MINI_PAINT_SET_VINTAGE_', 32),
 ('PACK_OF_72_RETROSPOT_CAKE_CASES', 31),
 ('PLASTERS_IN_TIN_CIRCUS_PARADE_', 52)]

In [228]:
#create list of freq_items_counts RDD
freq_items_counts_list = one_item_support.filter(lambda x: x[1]>=2).sortBy(lambda x: x[0]).collect()

In [229]:
freq_items_counts_list[:10]

[('DOLLY_GIRL_LUNCH_BOX', 35),
 ('JUMBO_BAG_RED_RETROSPOT', 31),
 ('LUNCH_BAG_APPLE_DESIGN', 43),
 ('LUNCH_BAG_RED_RETROSPOT', 50),
 ('LUNCH_BAG_SPACEBOY_DESIGN_', 42),
 ('LUNCH_BAG_WOODLAND', 39),
 ('LUNCH_BOX_WITH_CUTLERY_RETROSPOT_', 49),
 ('MINI_PAINT_SET_VINTAGE_', 32),
 ('PACK_OF_72_RETROSPOT_CAKE_CASES', 31),
 ('PLASTERS_IN_TIN_CIRCUS_PARADE_', 52)]

In [230]:
#create RDD with items in freq_items_count
freq_items = freq_items_counts.map(lambda x: x[0])
freq_items.take(10)

['DOLLY_GIRL_LUNCH_BOX',
 'JUMBO_BAG_RED_RETROSPOT',
 'LUNCH_BAG_APPLE_DESIGN',
 'LUNCH_BAG_RED_RETROSPOT',
 'LUNCH_BAG_SPACEBOY_DESIGN_',
 'LUNCH_BAG_WOODLAND',
 'LUNCH_BOX_WITH_CUTLERY_RETROSPOT_',
 'MINI_PAINT_SET_VINTAGE_',
 'PACK_OF_72_RETROSPOT_CAKE_CASES',
 'PLASTERS_IN_TIN_CIRCUS_PARADE_']

In [231]:
freq_items_list = freq_items_counts.map(lambda x: x[0]).collect()
freq_items_list[:10]

['DOLLY_GIRL_LUNCH_BOX',
 'JUMBO_BAG_RED_RETROSPOT',
 'LUNCH_BAG_APPLE_DESIGN',
 'LUNCH_BAG_RED_RETROSPOT',
 'LUNCH_BAG_SPACEBOY_DESIGN_',
 'LUNCH_BAG_WOODLAND',
 'LUNCH_BOX_WITH_CUTLERY_RETROSPOT_',
 'MINI_PAINT_SET_VINTAGE_',
 'PACK_OF_72_RETROSPOT_CAKE_CASES',
 'PLASTERS_IN_TIN_CIRCUS_PARADE_']

In [233]:
#function for creating item pair and frequency
def filter_and_pair(lst):
    ret=[]
    pairs=[]
    for i in lst:
        if i in freq_items_list:
            ret.append(i)
    ret.sort()
    for pair in itertools.combinations(ret,2):
        pairs.append((pair,1))
    return(pairs)

In [235]:
#create RDD for item pairs and their frequency
pair_counts = txt_RDD.flatMap(lambda x: filter_and_pair(x)).reduceByKey(add).sortBy( lambda x: x[0])
pair_counts.take(10)

[(('DOLLY_GIRL_LUNCH_BOX', 'JUMBO_BAG_RED_RETROSPOT'), 4),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_APPLE_DESIGN'), 7),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_RED_RETROSPOT'), 14),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_SPACEBOY_DESIGN_'), 14),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_WOODLAND'), 11),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BOX_WITH_CUTLERY_RETROSPOT_'), 12),
 (('DOLLY_GIRL_LUNCH_BOX', 'MINI_PAINT_SET_VINTAGE_'), 4),
 (('DOLLY_GIRL_LUNCH_BOX', 'PACK_OF_72_RETROSPOT_CAKE_CASES'), 3),
 (('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_CIRCUS_PARADE_'), 9),
 (('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_SPACEBOY'), 12)]

In [236]:
freq_pair_counts = pair_counts.filter(lambda x: x[1]>=2)
freq_pair_counts.take(10)
freq_pair_counts_list = freq_pair_counts.collect()
freq_pair_counts_list[:10]

[(('DOLLY_GIRL_LUNCH_BOX', 'JUMBO_BAG_RED_RETROSPOT'), 4),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_APPLE_DESIGN'), 7),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_RED_RETROSPOT'), 14),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_SPACEBOY_DESIGN_'), 14),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_WOODLAND'), 11),
 (('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BOX_WITH_CUTLERY_RETROSPOT_'), 12),
 (('DOLLY_GIRL_LUNCH_BOX', 'MINI_PAINT_SET_VINTAGE_'), 4),
 (('DOLLY_GIRL_LUNCH_BOX', 'PACK_OF_72_RETROSPOT_CAKE_CASES'), 3),
 (('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_CIRCUS_PARADE_'), 9),
 (('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_SPACEBOY'), 12)]

In [238]:
#create RDD for only item pairs
freq_pairs = freq_pair_counts.map(lambda x: x[0])
freq_pairs.take(10)

[('DOLLY_GIRL_LUNCH_BOX', 'JUMBO_BAG_RED_RETROSPOT'),
 ('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_APPLE_DESIGN'),
 ('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_RED_RETROSPOT'),
 ('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_SPACEBOY_DESIGN_'),
 ('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BAG_WOODLAND'),
 ('DOLLY_GIRL_LUNCH_BOX', 'LUNCH_BOX_WITH_CUTLERY_RETROSPOT_'),
 ('DOLLY_GIRL_LUNCH_BOX', 'MINI_PAINT_SET_VINTAGE_'),
 ('DOLLY_GIRL_LUNCH_BOX', 'PACK_OF_72_RETROSPOT_CAKE_CASES'),
 ('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_CIRCUS_PARADE_'),
 ('DOLLY_GIRL_LUNCH_BOX', 'PLASTERS_IN_TIN_SPACEBOY')]

In [239]:
#generate confidence for the item pair
conf_lst1=[]
for i in freq_pair_counts_list:
    conf1 = i[1]/(freq_items_counts.filter(lambda x: x[0]==i[0][0]).collect())[0][1]
    conf2 = i[1]/(freq_items_counts.filter(lambda x: x[0]==i[0][1]).collect())[0][1]
    conf_lst1.append(((i[0][0],i[0][1]),conf1))
    conf_lst1.append(((i[0][1],i[0][0]),conf2))

In [240]:
#sort items based on confidence value
conf_lst1.sort(key=lambda x: -x[1])
conf_lst1[:5]

[(('SET/6_RED_SPOTTY_PAPER_PLATES', 'SET/6_RED_SPOTTY_PAPER_CUPS'),
  0.9555555555555556),
 (('SET/6_RED_SPOTTY_PAPER_CUPS', 'SET/6_RED_SPOTTY_PAPER_PLATES'),
  0.9148936170212766),
 (('ROUND_SNACK_BOXES_SET_OF4_WOODLAND_', 'POSTAGE'), 0.8936170212765957),
 (('STRAWBERRY_LUNCH_BOX_WITH_CUTLERY', 'POSTAGE'), 0.8837209302325582),
 (('TEA_PARTY_BIRTHDAY_CARD', 'POSTAGE'), 0.8709677419354839)]